# Homework 4

For this assignment, you will be developing an artificial neural network to classify data given in the __[Dry Beans Data Set](https://archive.ics.uci.edu/ml/datasets/Dry+Bean+Dataset#)__. This data set was obtained as a part of a research study by Selcuk University, Turkey, in which a computer vision system was developed to distinguish seven different registered varieties of dry beans with similar features. More details on the study can be found in the following __[research paper](https://www.sciencedirect.com/science/article/pii/S0168169919311573)__. <br>
### **Make sure to use the lecture notebook on an introduction to keras and cross validation located [here](https://colab.research.google.com/drive/1ksEGL7SJ_wutCIyPYx7Loe5EPdOij6dJ?usp=sharing) and [here](https://colab.research.google.com/drive/1C9Mwf1J2ril1Q4l6n2BjQMb8YaFySG5_?usp=sharing)**.

## About the Data Set
Seven different types of dry beans were used in a study in Selcuk University, Turkey, taking into account the features such as form, shape, type, and structure by the market situation. A computer vision system was developed to distinguish seven different registered varieties of dry beans with similar features in order to obtain uniform seed classification. For the **classification** model, images of 13611 grains of 7 different registered dry beans were taken with a high-resolution camera. Bean images obtained by computer vision system were subjected to segmentation and feature extraction stages, and a total of 16 features - 12 dimensions and 4 shape forms - were obtained from the grains.

Number of Instances (records in the data set): __13611__

Number of Attributes (fields within each record, including the class): __17__

### Data Set Attribute Information:

1. __Area (A)__ : The area of a bean zone and the number of pixels within its boundaries.
2. __Perimeter (P)__ : Bean circumference is defined as the length of its border.
3. __Major axis length (L)__ : The distance between the ends of the longest line that can be drawn from a bean.
4. __Minor axis length (l)__ : The longest line that can be drawn from the bean while standing perpendicular to the main axis.
5. __Aspect ratio (K)__ : Defines the relationship between L and l.
6. __Eccentricity (Ec)__ : Eccentricity of the ellipse having the same moments as the region.
7. __Convex area (C)__ : Number of pixels in the smallest convex polygon that can contain the area of a bean seed.
8. __Equivalent diameter (Ed)__ : The diameter of a circle having the same area as a bean seed area.
9. __Extent (Ex)__ : The ratio of the pixels in the bounding box to the bean area.
10. __Solidity (S)__ : Also known as convexity. The ratio of the pixels in the convex shell to those found in beans.
11. __Roundness (R)__ : Calculated with the following formula: (4piA)/(P^2)
12. __Compactness (CO)__ : Measures the roundness of an object: Ed/L
13. __ShapeFactor1 (SF1)__
14. __ShapeFactor2 (SF2)__
15. __ShapeFactor3 (SF3)__
16. __ShapeFactor4 (SF4)__

17. __Classes : *Seker, Barbunya, Bombay, Cali, Dermosan, Horoz, Sira*__

In [3]:
!pip install scikeras
!pip install keras-tuner -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 9.2 MB/s eta 0:00:00


### Libraries that can be used :
- NumPy, SciPy, Pandas, Sci-Kit Learn, TensorFlow, Keras. You may also use PyTorch (though support may be limited)
- Any other library used during the lectures and discussion sessions.

### Other Notes
- Don't worry about not being able to achieve high accuracy, it is neither the goal nor the grading standard of this assignment.
- Discussion and Lecture materials should be helpful for doing the assignments.
- The homework submission should be a .ipynb file.


In [7]:
!git clone https://github.com/ucsd-cse151a-f24/hw4.git

Cloning into 'hw4'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 63 (delta 36), reused 7 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (63/63), 1.58 MiB | 20.43 MiB/s, done.
Resolving deltas: 100% (36/36), done.



## Exercise 1 : Building a Feed-Forward Neural Network(50 points)

### Exercise 1.1 : Data Preprocessing (10 points)

- As the classes are categorical, use one-hot encoding to represent the set of classes. You will find this useful when developing the output layer of the neural network.
- Split the data into training and testing set by __90:10__ and use the training set for training the model and the test set to evaluate the model performance. Please set verbose=0 to suppress output during training.
- Normalize each field of the input data using the min-max normalization technique.

__Notes:__

- Splitting of the dataset should be done __before__ the normalization step and __after__ the one-hot encoding.

In [18]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import keras
from keras import layers
from keras import ops
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, precision_score, recall_score
from sklearn.model_selection import RepeatedKFold, cross_val_score
from scikeras.wrappers import KerasClassifier
from sklearn.pipeline import Pipeline
import keras_tuner as kt
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping

In [13]:
df = pd.read_csv("hw4/Dry_Beans_Dataset.csv")
df = pd.get_dummies(df,columns=["Class"])
scaler = MinMaxScaler()
class_col = [col for col in df.columns if col.startswith("Class")]

X = df[[col for col in df.columns if col not in class_col]]
y = df[class_col]
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.1, random_state=42)

X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)


### Exercise 1.2 : Training and Testing the Neural Network (40 points)

Design a 3-layer (3 hidden layers and this does not include the input or output layer) artificial deep neural network, specifically a feed-forward multi-layer perceptron (using the sigmoid activation function), to classify the type of 'Dry Bean' given the other attributes in the data set, similar to the one mentioned in the paper above. Please note that this is a **multi-class classification** problem so select the right number of nodes accordingly for the input and output layers.

Consider the following hyperparameters while developing your model:

- Model type: Keras Sequential
- Make sure your input layer matches the size of your X matrix
- Number and type of hidden layers: 3 and Dense
- Number of nodes in each hidden layer: 12
- Learning rate should be 0.3
- Number of epochs should be 100
- The sigmoid function is to be used as the activation function in each layer
- Your output layer has to use a sigmoid function and the number of outputs should match the shape of your y
- Your loss function should be MSE
- Stochastic Gradient Descent should be used to minimize the error rate

**Note:** We are having you use MSE as your loss function for this model, is this a good choice? Why or why not? If not, what should you use instead in future models? Answer below

__Requirements once the model has been trained :__

- A confusion matrix for all classes, specifying the true positive, true negative, false positive, and false negative cases for each category in the class
- Since we do have OHE output (multi-class output) you will need to either reshape or argmax your outputs. Make sure they have already been thresholded as well i.e. look at yhat and do you see 1's and 0's?
- The accuracy and mean squared error (MSE) of the model
- The precision and recall for each label in the class

__Notes :__

- The mean squared error (MSE) values obtained __should be positive__.


In [14]:
import keras
from keras import layers
from keras import ops
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, precision_score, recall_score
from sklearn.model_selection import RepeatedKFold, cross_val_score
from scikeras.wrappers import KerasClassifier
from sklearn.pipeline import Pipeline
import keras_tuner as kt
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping

def create_model():
  model = Sequential([
      Dense(12, input_dim=X_train.shape[1], activation='sigmoid'),
      Dense(12, activation='sigmoid'),
      Dense(12, activation='sigmoid'),
      Dense(y_train.shape[1], activation='sigmoid')
  ])

  model.compile(
      optimizer=SGD(learning_rate=0.3),
      loss='mse',
      metrics=['mse']
  )

  history = model.fit(
      X_train_scaled, y_train,
      epochs=100,
      batch_size=32,
      verbose=0
  )

  loss, mse = model.evaluate(X_test_scaled, y_test, verbose=0)
  print(f"Test Loss: {loss}, Test MSE: {mse}")
  return model
model = create_model()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Test Loss: 0.0971468836069107, Test MSE: 0.0971468836069107


In [15]:
yhat_class = model.predict(X_test_scaled)
yhat_class = np.argmax(yhat_class,axis=1)
y_test_class = np.argmax(y_test, axis=1)
y_test_class

43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([0, 6, 3, ..., 3, 6, 0])

In [16]:

class_names = y_test.columns
cm = confusion_matrix(y_test_class, yhat_class)

print("\nMetrics for Each Class:")
for i, class_name in enumerate(class_names):
    TP = cm[i, i]
    FP = cm[:, i].sum() - TP
    FN = cm[i, :].sum() - TP
    TN = cm.sum() - (TP + FP + FN)

    print(f"{class_name}:")
    print(f"  True Positives (TP): {TP}")
    print(f"  True Negatives (TN): {TN}")
    print(f"  False Positives (FP): {FP}")
    print(f"  False Negatives (FN): {FN}")

accuracy = accuracy_score(y_test_class, yhat_class)
print(f"Accuracy: {accuracy:.2f}")

mse = mean_squared_error(y_test_class, yhat_class)
print(f"MSE: {mse:.2f}")

precision = precision_score(y_test_class, yhat_class, average=None)
recall = recall_score(y_test_class, yhat_class, average=None)

for i, (single_class,p, r) in enumerate(zip(class_names,precision, recall)):
    print(f"Class {single_class}: Precision = {p:.2f}, Recall = {r:.2f}")


Metrics for Each Class:
Class_BARBUNYA:
  True Positives (TP): 0
  True Negatives (TN): 1225
  False Positives (FP): 0
  False Negatives (FN): 137
Class_BOMBAY:
  True Positives (TP): 0
  True Negatives (TN): 1299
  False Positives (FP): 0
  False Negatives (FN): 63
Class_CALI:
  True Positives (TP): 0
  True Negatives (TN): 1167
  False Positives (FP): 0
  False Negatives (FN): 195
Class_DERMASON:
  True Positives (TP): 277
  True Negatives (TN): 599
  False Positives (FP): 421
  False Negatives (FN): 65
Class_HOROZ:
  True Positives (TP): 180
  True Negatives (TN): 951
  False Positives (FP): 230
  False Negatives (FN): 1
Class_SEKER:
  True Positives (TP): 189
  True Negatives (TN): 1097
  False Positives (FP): 65
  False Negatives (FN): 11
Class_SIRA:
  True Positives (TP): 0
  True Negatives (TN): 1118
  False Positives (FP): 0
  False Negatives (FN): 244
Accuracy: 0.47
MSE: 3.54
Class Class_BARBUNYA: Precision = 0.00, Recall = 0.00
Class Class_BOMBAY: Precision = 0.00, Recall = 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Exercise 2 : k-fold Cross Validation (20 points)

In order to avoid **using biased models**, use 10-fold cross validation to generalize the model from Ex1.2 on the given data set. You can choose a n_repeats value of 1-5

__Requirements :__
- Print the accuracy values during each iteration of the **cross validation** not the iterations per epoch or the epochs
- Print the overall average accuracy per each n_fold value, look at the documentation for the scoring parameter



In [17]:
from sklearn.model_selection import RepeatedKFold, cross_val_score
from scikeras.wrappers import KerasClassifier
from sklearn.pipeline import Pipeline
import keras_tuner as kt
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
keras_clf = KerasClassifier(model=create_model, epochs=100, batch_size=32, verbose=0)

n_splits = 10
n_repeats = 1
rkf = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)

pipeline = Pipeline([
    ('scaler', MinMaxScaler()),  # Scaling step
    ('classifier', keras_clf)
])

scores = cross_val_score(pipeline, X, np.argmax(y.values, axis=1), cv=rkf, scoring='accuracy')


print("Cross-Validation Accuracy for Each Fold:")
fold_accuracies = scores.reshape(n_repeats, n_splits)
for repeat_idx, fold_scores in enumerate(fold_accuracies):
    print(f"n_fold iteration {repeat_idx + 1}: {fold_scores}")
    print(f"Average accuracy for n_fold iteration {repeat_idx + 1}: {np.mean(fold_scores):.2f}")

overall_average_accuracy = np.mean(scores)
print(f"\nOverall Average Accuracy: {overall_average_accuracy:.2f}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Test Loss: 0.08600685000419617, Test MSE: 0.08600685000419617


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Test Loss: 0.0804283395409584, Test MSE: 0.0804283395409584


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

Test Loss: 0.08340685814619064, Test MSE: 0.08340685814619064


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Test Loss: 0.08439264446496964, Test MSE: 0.08439264446496964


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Test Loss: 0.08810669928789139, Test MSE: 0.08810669928789139
Cross-Validation Accuracy for Each Fold:
n_fold iteration 1: [0.28120411 0.30712711 0.43423953 0.22116091 0.41072741 0.2850845
 0.26304188 0.37178545 0.3232917  0.26230713]
Average accuracy for n_fold iteration 1: 0.32

Overall Average Accuracy: 0.32


## Exercise 3 : Hyperparameter Tuning (25 points)

Use either grid search or random search methodology to find the optimal number of nodes required in each hidden layer, as well as the optimal learning rate and the different activation functions or optimization approaches, [keras_tuner examples](https://keras.io/guides/keras_tuner/getting_started/) such that the accuracy of the model is maximum for the given data set.

__Requirements :__
- The set of optimal hyperparameters
- Try your best to maximize accuracy using this set of optimal hyperparameters

__Note :__ Hyperparameter tuning takes a lot of time to execute. Make sure that you choose the appropriate number of each hyperparameter (preferably 3 of each), and that you allocate enough time to execute your code. Make sure to tune at least three parameters with three options each at a minimum, but feel free to experiment with more, just recognize that it will grow exponentially in running time

In [20]:
from sklearn.preprocessing import MinMaxScaler
from keras.utils import to_categorical
import keras_tuner as kt


def build_model(hp):
    model = Sequential()
    model.add(Dense(
        hp.Int('units_layer1', min_value=8, max_value=128, step=40),
        activation=hp.Choice('activation_layer1', ['relu', 'tanh', 'sigmoid']),
        input_dim=X_train.shape[1]
    ))
    model.add(Dense(
        hp.Int('units_layer2', min_value=8, max_value=128, step=40),
        activation=hp.Choice('activation_layer2', ['relu', 'tanh', 'sigmoid'])
    ))
    model.add(Dense(
        hp.Int('units_layer3', min_value=8, max_value=128, step=40),
        activation=hp.Choice('activation_layer3', ['relu', 'tanh', 'sigmoid'])
    ))
    model.add(Dense(y_train.shape[1], activation='softmax'))
    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', [0.001, 0.01, 0.1])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,
    executions_per_trial=1,
    directory='hyperparameter_tuning',
    project_name='keras_tuner_pipeline'
)

tuner.search(X_train_scaled, y_train, epochs=50, validation_split=0.2, verbose=0)


best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("\nOptimal Hyperparameters:")
print(f" - Units in Layer 1: {best_hps.get('units_layer1')}")
print(f" - Activation Function in Layer 1: {best_hps.get('activation_layer1')}")
print(f" - Units in Layer 2: {best_hps.get('units_layer2')}")
print(f" - Activation Function in Layer 2: {best_hps.get('activation_layer2')}")
print(f" - Learning Rate: {best_hps.get('learning_rate')}")

# Build the best model
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
best_model = tuner.hypermodel.build(best_hps)

# Train the best model
history = best_model.fit(X_train_scaled, y_train, epochs=50, validation_split=0.2, verbose=0, callbacks=[early_stopping])

# Evaluate the best model
test_loss, test_accuracy = best_model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"\nTest Accuracy: {test_accuracy:.2f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/lo


Optimal Hyperparameters:
 - Units in Layer 1: 48
 - Activation Function in Layer 1: relu
 - Units in Layer 2: 128
 - Activation Function in Layer 2: relu
 - Learning Rate: 0.001


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Test Accuracy: 0.93


## Exercise 4 - Collaborative Statement (5 points)

It is mandatory to include a Statement of Collaboration in each submission, that follows the guidelines below.
Include the names of everyone involved in the discussions (especially in-person ones), and what was discussed.
All students are required to follow the academic honesty guidelines posted on the course website. For
programming assignments in particular, I encourage students to organize (perhaps using Piazza) to discuss the
task descriptions, requirements, possible bugs in the support code, and the relevant technical content before they
start working on it. However, you should not discuss the specific solutions, and as a guiding principle, you are
not allowed to take anything written or drawn away from these discussions (no photographs of the blackboard,
written notes, referring to Piazza, etc.). Especially after you have started working on the assignment, try to restrict
the discussion to Piazza as much as possible, so that there is no doubt as to the extent of your collaboration.

Even if you did not use any outside resources or collaborate with anyone, please state that explicitly in the space below.

Bobby Zhu